In [26]:
import mesa
import pandas as pd
import numpy as np
from mesa import Model, Agent
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random

In [2]:
class Person(Agent):

    def __init__(self, model, talent, capital = 1):

        super().__init__(model)

        self.talent = talent
        self.capital = capital



    def live(self):

        # check cell and neighborhood for events
        pos_events = [event for event in self.model.grid.iter_neighbors(self.pos, moore = True, include_center = True) if isinstance(event, PositiveEvent)]
        neg_events = [event for event in self.model.grid.iter_neighbors(self.pos, moore = True, include_center = True) if isinstance(event, NegativeEvent)]

        # double capital with probability proportional to talent if positive event was encountered
        if pos_events:

            r = random.random()
            
            if r < self.talent:
                self.capital = 2 * self.capital

        # halve capital if negative event was encountered
        if neg_events:
            
            self.capital = 0.5 * self.capital


In [3]:
class PositiveEvent(Agent):

    def __init__(self, model):
        super().__init__(model)

    def move(self):

        # move to random cell
        new_pos = random.choice(list(self.model.grid.coord_iter()))[1]
        self.model.grid.move_agent(self, new_pos)

In [4]:
class NegativeEvent(Agent):

    def __init__(self, model):
        super().__init__(model)

    def move(self):

        # move to random cell
        new_pos = random.choice(list(self.model.grid.coord_iter()))[1]
        self.model.grid.move_agent(self, new_pos)

In [ ]:
class LuckTalentModel(Model):

    def __init__(self, height = 20,
                 width = 20, 
                 n_actors = 100,
                 n_events = 10,
                 mean_talent = 0.5, 
                 sd_talent = 0.1,
                 seed = None):

        super().__init__(seed=seed)

        self.height = height
        self.width = width
        self.n_actors = n_actors
        self.n_events = n_events

        self.years_passed = 0

        self.grid = MultiGrid(width=self.width, height=self.height, torus = True)

        # create agents 
        for agent in range(n_actors):

            # sample talent from normal distribution with given mean and sd
            talent = np.random.normal(loc = mean_talent, scale=sd_talent)
            # choose location randomly
            coords = random.sample(list(self.grid.empties), 1)[0]

            agent = Person(self, talent)
            self.grid.place_agent(agent, coords)


        # create events
        n_pos = int(self.n_events / 2)
        n_neg = n_events - n_pos

        for pos_event in range(n_pos):

            # select random cell
            coords = random.choice(list(self.grid.coord_iter()))[1]
            event = PositiveEvent(self)
            self.grid.place_agent(event, coords)

        for neg_event in range(n_neg):

            # select random cell
            coords =  random.choice(list(self.grid.coord_iter()))[1]
            event = NegativeEvent(self)
            self.grid.place_agent(event, coords)


        self.running = True


    def step(self):

        self.agents_by_type[Person].do("live")
        self.agents_by_type[NegativeEvent].do("move")
        self.agents_by_type[PositiveEvent].do("move")

        self.years_passed += 0.5

        
        # end model if 40 years have passed
        if self.years_passed >= 40:
            for agent in self.agents_by_type[Person]:
                print(agent.capital)
                
            self.running = False

            

In [13]:
firstModel = LuckTalentModel(20, 20, 100, 10, 0.5, 0.2)

while firstModel.running:
    firstModel.step()

0.25
0.03125
1.0
16.0
0.0625
0.25
2.0
0.015625
0.125
0.125
0.015625
0.5
0.0625
0.03125
1.0
0.5
0.015625
0.0078125
0.015625
0.125
0.03125
8.0
0.125
0.25
0.125
0.0001220703125
32.0
2.0
0.0078125
0.00390625
0.0078125
0.015625
2.0
0.0625
0.5
4.0
0.0625
0.03125
0.125
0.25
0.25
4.0
0.0625
0.5
0.0078125
0.0009765625
0.125
4.0
2.0
1.0
0.0078125
0.0078125
0.0078125
0.00390625
0.00390625
0.125
0.000244140625
0.0009765625
0.0625
8.0
0.015625
0.0009765625
0.015625
0.0625
0.0078125
16.0
0.00390625
0.03125
0.25
0.125
0.00048828125
0.0625
8.0
0.00390625
0.25
0.5
0.03125
0.5
8.0
0.015625
0.03125
1.0
0.001953125
1.0
0.015625
0.001953125
8.0
0.0625
2.0
2.0
0.5
1.0
0.00390625
0.00390625
0.00390625
2.0
0.000244140625
0.5
0.03125
0.015625


In [25]:
from mesa.visualization import (
    Slider,
    SolaraViz,
    make_plot_component,
    make_space_component,
)

def agent_portrayal(agent):

    if agent is None:
        return
    
    portrayal = {"size": 25}

    if isinstance(agent, Person):
        portrayal["color"] = "black"
        portrayal["marker"] = "x"
    elif isinstance(agent, NegativeEvent):
        portrayal["color"] = "red"
        portrayal["marker"] = "o"
    elif isinstance(agent, PositiveEvent):
        portrayal["color"] = "green"
        portrayal["marker"] = "o"
        
    return portrayal

# model_params = {"width":20, "height":20, "n_actors":100, "n_events":10, "mean_talent":0.5, "sd_talent": 0.3}

model_params = {
    "seed": {
        "type": "InputText",
        "value": 42,
        "label": "Random Seed",
    },
    "height": Slider(label="Height of grid", value = 20, min = 5, max = 30, step = 1),
    "width": Slider("Width of grid", min = 5,value = 20, max = 30, step = 1),
    "n_events": Slider("Number of events", value = 20, min = 10, max = 100, step = 2),
    "n_actors": Slider("Number of People",value = 100, min = 10, max = 500, step = 1),
    "mean_talent": Slider("Mean Talent", value = 0.5, min = 0, max = 1, step = 0.1),
    "sd_talent": Slider("SD Talent", value = 0.1 , min = 0, max = 1, step = 0.1)
}

space_component = make_space_component(agent_portrayal)



page = SolaraViz(
    LuckTalentModel(),
    components=[space_component],
    model_params=model_params,
    name="Talent vs. Luck Model"
)

page

Cannot show ipywidgets in text

0.00048828125
4.76837158203125e-07
0.00048828125
0.00048828125
4.76837158203125e-07
0.00048828125
5.960464477539063e-08
6.103515625e-05
7.62939453125e-06
1.0
0.001953125
7.62939453125e-06
6.103515625e-05
6.103515625e-05
0.25
0.001953125
3.814697265625e-06
7.62939453125e-06
0.0078125
0.0009765625
3.0517578125e-05
1.52587890625e-05
0.0001220703125
1.52587890625e-05
6.103515625e-05
0.000244140625
1.52587890625e-05
0.125
0.00390625
0.0625
0.000244140625
0.0009765625
0.001953125
0.0001220703125
0.0001220703125
0.00048828125
0.00390625
0.25
0.5
0.125
0.0009765625
0.000244140625
1.52587890625e-05
0.0009765625
0.0009765625
0.001953125
0.00390625
0.0078125
1.9073486328125e-06
0.00390625
0.00048828125
0.125
0.0001220703125
0.00048828125
0.0009765625
0.0001220703125
0.03125
0.00390625
0.001953125
1.9073486328125e-06
3.0517578125e-05
0.001953125
3.0517578125e-05
0.015625
3.0517578125e-05
0.00048828125
0.001953125
6.103515625e-05
0.0078125
0.0001220703125
0.0078125
3.0517578125e-05
0.0625
0.0019531